In [ ]:
from eComply.eComply import eComply
from ParksGIS.ParksGIS import GISFactory, Server, LayerServerGen, LayerQuery, LayerEdits
from pandas import DataFrame
# from Transformer import Transformer

Filters

In [ ]:
from datetime import datetime
from typing import Any, Callable, Literal, Optional
from pandas import merge, Series


def to_json(obj: Any) -> str:
    return obj.to_json(orient="records", date_format="iso")


def apply_edits(dict: dict) -> dict:
    if dict["edits"] is not None and 0 < len(dict["edits"]):
        try:
            result = dict["repo"].apply_edits(dict["edits"])
            print(f"Edits Applied: {len(result)}")
        except Exception as e:
            print(f"apply_edits error: {e}")

    return dict


def guard_edits(
    dict: dict,
    key: str,
    func: Callable[[dict], Any],
) -> dict | None:
    edits = dict.get("edits", [])
    if 0 == len(edits):
        return dict

    edits = edits[0]
    ids = edits[~edits[key].isna()][key]
    if 0 == len(ids):
        return dict

    try:
        func(dict)
        return dict
    except Exception as e:
        print(e)
        return None


def extract_changes(
    repo: Any,
    server_gen: int,
    layerId: int,
    func: Callable[[Any, Any], Optional[DataFrame]],
) -> dict[str, Any]:
    dict = {"edits": [], "server_gen": server_gen}

    try:
        changes = repo.extract_changes([LayerServerGen(layerId, server_gen)])
        dict["serverGen"] = changes["layerServerGens"][0]["serverGen"] // 1000

        objectIds = (
            changes["edits"][0]["objectIds"]["adds"]
            + changes["edits"][0]["objectIds"]["updates"]
        )
        if 0 < len(objectIds):
            result = func(repo, objectIds)
            if result is not None:
                dict["edits"].append(result)
        else:
            print(f"No changes from layer {layerId}")
    except Exception as e:
        print(f"extract_changes error: {e}")
    finally:
        return dict


def get_changes(
    service: Any,
    serverGen: int,
    func: Callable[
        [
            Any,
            datetime,
        ],
        LayerEdits,
    ],
) -> dict[str, Any]:
    dict = {"edits": []}
    try:
        result = func(
            service,
            datetime.fromtimestamp(serverGen),
        )
        if result is not None:
            dict["edits"].append(result)
    except Exception as e:
        print(f"get_changes error: {e}")
    finally:
        return dict


def get_edits(dict: dict) -> DataFrame:
    return dict["edits"][0]


def set_edits(dict: dict, data: DataFrame):
    dict["edits"][0] = data


def join(
    list: Series | list,
    guid: bool = False,
) -> str:
    if guid:
        return "'" + "','".join(str(i) for i in list) + "'"
    else:
        return ",".join(str(i) for i in list)


def pipeline(
    dict: dict[str, Any],
    *funcs: Callable[
        [dict[str, Any]],
        dict[str, Any],
    ],
):
    if not funcs:
        raise ValueError("At least one function must be provided.")

    result = dict
    for step, func in enumerate(funcs):
        if result is None:
            print(f"**Pipeline end at step {step}**")
            break

        if not isinstance(func, Callable):
            raise TypeError(f"Expected a callable for step {step}")

        result = func(result)
    return result


#######################################################################################


# Sending
def contract_extract_changes(dict: dict) -> dict:
    layerId = 1

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    ["*"],
                    f"OBJECTID IN ({join(objectIds)}) AND EcomplyContract = 1",
                )
            ]
        )[layerId]

        print(f"Contracts Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict.update(
        extract_changes(
            dict["repo"],
            dict["server_gens"].at[0, "Contract"],
            layerId,
            strategy,
        )
    )

    return dict


def contract_send_edits(dict: dict) -> dict:
    def strategy(dict):
        edits = get_edits(dict)
        # print(to_json(edits))

        response = dict["service"].post_contracts(to_json(edits))
        print(f"Contracts result: {response}")

    return guard_edits(dict, "OBJECTID", strategy)


def work_order_extract_changes(dict: dict) -> dict:
    layerId = 0

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    [
                        "InspectionGlobalID",
                        "Type",
                        "Status",
                        "LocationDetails",
                        "ActualFinishDate",
                        "Comments",
                        "Contract",
                        "CancelReason",
                        "GlobalID",
                        "ClosedDate",
                        "ClosedByERN",
                        "ClosedByName",
                        "CancelDate",
                        "CancelByERN",
                        "CancelByName",
                        "CreatedDate",
                        "CreatedBYERN",
                        "CreatedByName",
                        "UpdatedDate",
                        "UpdatedByERN",
                        "UpdatedByName",
                        "WOEntity",
                        "PROJSTARTDATE",
                        "Project",
                        "RecommendedSpecies",
                        "ClosedBySystem",
                        "OBJECTID",
                    ],
                    f"OBJECTID IN ({join(objectIds)})",
                )
            ]
        )[layerId].rename(
            columns={
                "LocationDetails": "Location",
                "GlobalID": "WorkOrderGlobalId",
                "PROJSTARTDATE": "ProjStartDate",
                "RecommendedSpecies": "RecSpecies",
                "OBJECTID": "ObjectId",
            }
        )

        print(f"Work Orders Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict.update(
        extract_changes(
            dict["repo"],
            dict["server_gens"].at[0, "WorkOrder"],
            layerId,
            strategy,
        )
    )

    return dict


def wo_get_associated_planting_space_globalid(dict: dict) -> dict:
    key = "InspectionGlobalID"

    def strategy(dict):
        layerId = 4

        edits = get_edits(dict)
        inspections = (
            dict["repo"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        ["PlantingSpaceGlobalID", "GlobalID"],
                        f"GlobalID IN ({join(edits[~edits[key].isna()][key], True)})",
                    )
                ]
            )[layerId]
            .rename(columns={"GlobalID": key})
        )

        edits = merge(edits, inspections, on=key, how="left")
        del edits[key]

        set_edits(dict, edits)
        # print(to_json(edits[:1]))
        print(f"Planting Space Ids found: {len(inspections)}")

    return guard_edits(dict, key, strategy)


def wo_get_associated_planting_space(dict: dict) -> dict:
    key = "PlantingSpaceGlobalID"

    def strategy(dict):
        layerId = 2

        edits = get_edits(dict)
        plantingSpaces = (
            dict["repo"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "ParkName",
                            "ParkZone",
                            "Borough",
                            "CommunityBoard",
                            "BuildingNumber",
                            "StreetName",
                            "CityCouncil",
                            "StateAssembly",
                            "GISPROPNUM",
                            "CrossStreet1",
                            "CrossStreet2",
                            "PlantingSpaceOnStreet",
                            "ObjectID",
                            "GlobalID",
                        ],
                        f"GlobalID IN ({join(edits[~edits[key].isna()][key], True)})",
                    )
                ]
            )[layerId]
            .rename(
                columns={
                    "GlobalID": "PlantingSpaceGlobalID",
                    "OBJECTID": "PlantingSpaceId",
                    "PlantingSpaceOnStreet": "OnStreetSite",
                }
            )
        )

        edits = merge(edits, plantingSpaces, on=key, how="left")

        set_edits(dict, edits)
        # print(to_json(edits[:1]))
        print(f"Planting Spaces hydrated: {len(plantingSpaces)}")

    return guard_edits(dict, key, strategy)


def work_order_send_edits(dict: dict) -> dict:
    def strategy(dict):
        edits = get_edits(dict)
        # print(to_json(edits))

        response = dict["service"].post_work_orders(to_json(edits))
        print(f"Work Orders result: {response}")

    return guard_edits(dict, "PlantingSpaceGlobalID", strategy)


# Receiving
def work_order_get_changes(dict: dict) -> dict:
    layerId = 0

    def strategy(service, fromDateTime: datetime):
        changes = service.get_work_orders(fromDateTime)
        apply_edits = LayerEdits(layerId, updates=changes)
        print(f"WorkOrders Recieved: {len(changes)}")
        return apply_edits

    dict.update(
        get_changes(
            dict["service"],
            dict["server_gens"].at[0, "WorkOrder"],
            strategy,
        )
    )

    return dict


def wo_update_associated_inspection(dict: dict) -> dict:
    layerId = 4
    key = "WorkOrderGlobalID"

    def strategy(dict):
        edits = get_edits(dict)

        inspections = (
            dict["repo"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "InspectionGlobalID",
                            "HasActiveWorkOrder",
                        ],
                        f"{key} IN ({join(edits[key], True)})",
                    )
                ]
            )[layerId]
            .rename(columns={"PlantingSpaceGlobalID": "plantingSpaceGlobalId"})
        )

        edits = merge(inspections, edits, on=key, how="left")
        edits.loc[
            edits["Status"] == Literal["Closed", "Canceled"], "HasActiveWorkOrder"
        ] = 0

        dict["edits"].append(
            LayerEdits(
                layerId, updates=edits["InspectionGlobalID", "HasActiveWorkOrder"]
            )
        )

        print(f"Inspections To Update: {len(inspections)}")

    return guard_edits(dict, key, strategy)


def wo_update_associated_platingSpace(dict: dict) -> dict:
    layerId = 2

    def strategy(dict):
        edits = get_edits(dict)

        plantingSpaces = (
            dict["repo"].query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "GlobalID",
                            "BuildingNumber",
                            "StreetName",
                            "CrossStreet1",
                            "CrossStreet2",
                        ],
                        f"GlobalID IN ({join(edits['plantingSpaceGlobalId'], True)})",
                    )
                ]
            )[layerId]
            # .rename(
            #     columns={
            #         "GlobalID": "plantingSpaceGlobalId",
            #     }
            # )
        )

        # Transformer.update(
        #     plantingSpaces,
        #     edits,
        #     "plantingSpaceGlobalId",
        #     {
        #         "BuildingNumber": {"Source": "BuildingNumber"},
        #         "StreetName": {"Source": "StreetName"},
        #         "CrossStreet1": {"Source": "CrossStreet1"},
        #         "CrossStreet2": {"Source": "CrossStreet2"},
        #     },
        # )

        dict["edits"].append(LayerEdits(layerId, updates=plantingSpaces))

        print(f"Planting Spaces To Update: {len(plantingSpaces)}")

    return guard_edits(dict, "plantingSpaceGlobalId", strategy)


def work_order_line_item_get_changes(dict: dict) -> dict:
    layerId = 2

    def strategy(
        service,
        fromDateTime: datetime,
    ):
        changes = service.get_work_order_line_items(fromDateTime)
        apply_edit = LayerEdits(layerId, updates=changes)
        print(f"Line Items Recieved: {len(changes)}")
        return apply_edit

    dict.update(
        get_changes(
            dict["service"],
            dict["server_gens"].at[0, "WorkOrder"],
            strategy,
        )
    )

    return dict


def contract_get_changes(dict: dict) -> dict:
    layerId = 1

    def strategy(service, fromDateTime: datetime):
        changes = service.get_contracts(fromDateTime)
        apply_edits = LayerEdits(layerId, updates=changes)
        print(f"Contracts Recieved: {len(changes)}")
        return apply_edits

    dict.update(
        get_changes(
            dict["service"],
            dict["server_gens"].at[0, "Contract"],
            strategy,
        )
    )

    return dict

Pipeline

In [ ]:
proxy = "@bcpxy.nycnet:8080"

import os

# Set proxy incase environment not set
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
# bypass proxy on parks domains
os.environ["NO_PROXY"] = ".parks.nycnet"

# arc_gis = "https://formsgisportal.parks.nycnet"
# arc_gis = 'https://stg-formsgisportal.parks.nycnet'
arc_gis = "https://dev-formsgisportal.parks.nycnet"

factory = GISFactory(
    url=arc_gis + "/portal/home",
    username="forms.python_user",
    password="formsPython24*",
)
e_comply_repo = factory.create_feature(
    arc_gis + "/server/rest/services/eComply/eComplyContract/FeatureServer"
)
data_push_repo = factory.create_feature(
    arc_gis + "/server/rest/services/DataPush/ForMSDataPush/FeatureServer"
)

e_comply = eComply(
    url="https://nycparks-stage.ecomply.us/WebAPI",
    username="ff@ecomply.us",
    password="!test123",
)

# Push DomainValues
data_push_repo = factory.create_feature(
    arc_gis + "server/rest/services/DataPush/ForMSDataPush/FeatureServer"
)
domain_values = data_push_repo.query_domains(
    [
        LayerDomainNames(
            0,
            [
                "WOContract",
                "WOProject",
                "WOEntity",
                "WOStatus",
                "WOCategory",
                "WOType",
                "GenusSpecies",
            ],
        )
    ]
)
domains = [
    {
        "domainName": domain["name"],
        "code": str(value["code"]),
        "value": value["name"],
    }
    for domain in domain_values
    for value in domain["codedValues"]
]
e_comply.post_domain_values(domains)
print("DomainValues Pushed")

import time

seconds = 60 * 60 * 24
epoch = time.time() - seconds
server_gens = DataFrame({"Contract": [epoch], "WorkOrder": [epoch]})

# server_gens = e_comply_repo.query([LayerQuery(3, ["*"])])[3]
updated_server_gens = server_gens.copy()
print(updated_server_gens)

print()
send_contracts = pipeline(
    {
        "service": e_comply,
        "repo": e_comply_repo,
        "server_gens": updated_server_gens,
    },
    contract_extract_changes,
    contract_send_edits,
)
updated_server_gens["Contract"] = send_contracts["server_gen"]
print(updated_server_gens)
e_comply_repo.apply_edits(
    [LayerEdits(3, updates=updated_server_gens.to_json(orient="records"))]
)

print()
send_work_orders = pipeline(
    {
        "service": e_comply,
        "repo": data_push_repo,
        "server_gens": updated_server_gens,
    },
    work_order_extract_changes,
    wo_get_associated_planting_space_globalid,
    wo_get_associated_planting_space,
    work_order_send_edits,
)
updated_server_gens["WorkOrder"] = send_work_orders["serverGen"]
print(updated_server_gens)
e_comply_repo.apply_edits(
    [LayerEdits(3, updates=updated_server_gens.to_json(orient="records"))]
)

print()
apply_work_orders = pipeline(
    {
        "service": e_comply,
        "repo": data_push_repo,
        "server_gens": server_gens,
    },
    work_order_get_changes,
    wo_update_associated_inspection,
    wo_update_associated_platingSpace,
    apply_edits,
)

print()
apply_work_order_line_items = pipeline(
    {
        "service": e_comply,
        "repo": e_comply_repo,
        "server_gens": server_gens,
    },
    work_order_line_item_get_changes,
    apply_edits,
)

print()
apply_contracts = pipeline(
    {
        "service": e_comply,
        "repo": e_comply_repo,
        "server_gens": server_gens,
    },
    contract_get_changes,
    apply_edits,
)